In [1]:
# !pip install bs4
# !pip install lxml
# !pip install requests

In [2]:
import os
import sys
import re
import requests
import time

from bs4 import BeautifulSoup
from pprint import pprint

In [3]:
url_basic_base = 'http://movie.naver.com/movie/bi/mi/basic.nhn?code=%s'

In [4]:
movie_id = 150198 # 너의 이름은

In [5]:
url = url_basic_base % movie_id
req = requests.get(url)
html = req.text

## parsing
data = BeautifulSoup(html, "html.parser") # data = BeautifulSoup(html, 'lxml')

# scraping test

In [6]:
origin_title = data.select('div[class=mv_info] strong[class=h_movie2]')
origin_title

[<strong class="h_movie2" title="君の名は。									, your name.										, 					2016">君の名は。
 				
 					, your name.
 					
 					, 
 					2016</strong>,
 <strong class="h_movie2" title="君の名は。, your name., 2016">君の名は。, your name., 2016</strong>]

In [7]:
origin_title[0].text

'君の名は。\r\n\t\t\t\t\r\n\t\t\t\t\t, your name.\r\n\t\t\t\t\t\r\n\t\t\t\t\t, \r\n\t\t\t\t\t2016'

In [8]:
origin_title[0].text.replace("\t", '').replace('\r', '').replace('\n', '')

'君の名は。, your name., 2016'

In [9]:
kor_title = data.select('div[class=mv_info] h3[class=h_movie] a')
kor_title[0].text

'너의 이름은.'

## try - exception practice

# Scraping

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def get_soup(url):
    try:
        r = requests.get(url, headers=headers).text
        return BeautifulSoup(r, 'lxml')
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback_details = {
                     'filename': exc_traceback.tb_frame.f_code.co_filename,
                     'lineno'  : exc_traceback.tb_lineno,
                     'name'    : exc_traceback.tb_frame.f_code.co_name,
                     'type'    : exc_type.__name__,
                     'message' : str(e)
                    }
        pprint(traceback_details)
        return ''

# Basic pages
def get_basic_page(movie_id):
    url = url_basic_base % movie_id
    return get_soup(url)

def parse_basic_page(page):
    movie = {}

    score = _parse_main_score(page)
    movie['expert_score'] = score[0]
    movie['netizen_score'] = score[1]

    movie['title'] = _parse_title(page)
    movie['e_title'] = _parse_e_title(page)

    try:
        basic_inf = page.select('dl[class=info_spec]')[0]
        movie['genres'] = _parse_genres(page)
        movie['countries'] = _parse_countries(page)
        movie['running_time'] = _parse_running_time(page)
        movie['open_dates'] = _parse_open_date(page)
        movie['grade'] = _parse_grade(page)
        return movie
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback_details = {
                     'filename': exc_traceback.tb_frame.f_code.co_filename,
                     'lineno'  : exc_traceback.tb_lineno,
                     'name'    : exc_traceback.tb_frame.f_code.co_name,
                     'type'    : exc_type.__name__,
                     'message' : str(e)
                    }
        return movie
    
def _parse_title(page):
    try: 
        return page.select('div[class=mv_info] h3[class=h_movie] a')[0].text.strip()
    except:
        return ''

def _parse_e_title(page):
    try:
        return page.select('div[class=mv_info] strong[class=h_movie2]')[0].text.replace('\r', '').replace('\t', '').replace('\n', '').strip()
    except:
        return ''

def _parse_genres(page):
    genres = page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?genre=]')
    return list({genre.text for genre in genres})
    
def _parse_countries(page):
    countries = page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?nation=]')
    return list({country.text for country in countries})
    
def _parse_running_time(page):
    running_time = 0
    try:
        running_time = re.search(r"\d+분", page.text).group()[:-1]
    except:
        running_time = 0
    return running_time

def _parse_open_date(page):
    return list({d for d in re.findall(r"\d+\.\d+\.\d+ 재*개봉", page.text)})

def _parse_grade(page):
    try:
        return page.select('a[href^=/movie/sdb/browsing/bmovie.nhn?grade]')[0].text
    except:
        return ''

def _parse_main_score(page):
    try:
        main_score = page.select('div[class=main_score]')[0]
        expert_score = main_score.select('div[class=spc_score_area] div[class=star_score]')[0].text.replace('\n','')
        netizen_score = main_score.select('div[class=score] div[class=star_score] span[class=st_off]')[0].text.replace('관람객 평점 ', '').replace('점', '')
        return expert_score, netizen_score
    except Exception as e:
        # print('error from _parse_main_score', e)
        return -1, -1

# def _parse_poster(page):
#     try:
#         return page.select('img[class=_Img]')[0]
#     except:
#         return ''

# scraping TIPS 

### 1. time.sleep

In [11]:
for movie_id in range(134960, 134965):
    url = url_basic_base % movie_id
    page = get_soup(url)
    title = _parse_title(page)
    print('%d: %s' % (movie_id, title))
    time.sleep(1.0) # 1초 기다리기

134960: 그놈이다
134961: 어 라 말라
134962: 아이
134963: 라라랜드
134964: 콜 포 헬프


### 2. User agent

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get(url, headers=headers)

http://www.useragentstring.com/pages/useragentstring.php?name=Chrome

### 3. file download

파이썬에서도 음악/사진 파일들을 다운로드 받을 수 있습니다. 다운로드라는 것도 서버와 내 컴퓨터 간의 통로를 열어두고, 전송이 되는 byte 정보들을 모아서 다시 음악, 사진 포멧으로 읽는 것입니다. 물론 스트리밍 서비스들은 temporal하게 내 컴퓨터에 데이터가 쌓이지 않게 막을 수도 있습니다. 그런 종류가 아니라, \<a>라는 태그로 링크가 걸려있는 이미지 파일들을 다운로드 해보겠습니다. 

아래 코드는 urllib.request.urlopen을 통하여 서버와 내 컴퓨터 간의 통신 통로를 열어둡니다. while loop 안에서 열려진 통로에서 100000000 byte만큼의 정보를 가져와 buffer에 넣습니다. 그리고 이 정보를 미리 열어둔 downloaded_file이라는 파일에 적습니다. 주고 받은 정보가 텍스트가 아니라 바이트이기 때문에 'wb'로 파일을 열어둡니다. 다운로드가 모두 끝나면 열어둔 통신 서버를 닫고
    
    opened.close()
    
열어둔 파일도 닫습니다. 

    downloaded_file.close()
    
다운로드가 성공적으로 되었다면 True가, 실패했다면 False가 return 되도록 try - except로 이 코드 부분을 감싸줍니다

In [12]:
import urllib

def download_file(url, fname):
    try:
        downloaded_file = open(fname, "wb")
        opened = urllib.request.urlopen(url)
        while True:
            buffer = opened.read(100000000)
            if len(buffer) == 0:
                break
            downloaded_file.write(buffer)
        opened.close()
        downloaded_file.close()
        return True
    except:
        return False

In [16]:
movie_img_url = page.select('img[class=_Img]')
movie_img_url

[<img alt="POSTER" class="_Img" src="http://movie.phinf.naver.net/20150123_72/14220061096862xbEP_JPEG/movie_image.jpg?type=m427_320_2"/>]

In [17]:
download_file('http://movie.phinf.naver.net/20161125_80/1480051484421z6wDD_JPEG/movie_image.jpg', 'movie_image.jpg')

True

In [18]:
%ls

1_data_file_json.ipynb                data/
2_pos_tag_with_konlpy.ipynb           google_logo.png
3_scraping_naver_movie.ipynb          movie_image.jpg
5_Keyword_extraction.ipynb            mypy/
6_corpus_class.ipynb                  soy/
README.md                             word-movers-distance-in-python.ipynb
WMD.pdf                               강의자료/
